In [ ]:
!pip install python-chess
import chess
import sys
import json
from pathlib import Path
import hashlib
import os
import getpass
import time
import random
import math
import socket # For conceptual online mode
import threading

# --- Configuration ---
USER_DATA_DIR = Path.home() / ".chess_club"  # Persistent cross-platform directory
USER_DATA_FILE = USER_DATA_DIR / "chess_users.json"
LAST_LOGIN_FILE = USER_DATA_DIR / "last_login.txt"
USER_DATA_DIR.mkdir(parents=True, exist_ok=True)
AI_SIMULATION_MODE = False  # Add at top with other configs
SIMULATION_SPEED_CUTOFF = 100  # x speed threshold for simplified logic
INITIAL_ELO = 200
AI_EASY_BASE_ELO = 600      # EJ-1.0
AI_MEDIUM_BASE_ELO = 800    # EJ-1.1
AI_ADVANCED_BASE_ELO = 1200 # EJ-Bot 1.2
AI_MASTER_BASE_ELO = 2000   # EJ-Bot 1.3
MIN_ELO = 100  # Add this to prevent negative ratings
K_FACTOR = 16
MAX_GRANDMASTER_ELO = 4000
AI_LEARNING_POINTS_FILE = "ai_learning_points.json"
SIMULATION_TIME_LIMIT_EASY = 0.000001 # Slightly increased base time
SIMULATION_ADVANCED_BASE_TIME_LIMIT_SIMULATION = 0.005 # Slightly increased base time
SIMULATION_TIME_LIMIT_ADVANCED = 0.05 # Quicker
SIMULATION_INITIAL_DEPTH_ADVANCED = 3 # Kept at 3 for now
AI_EASY_GAME_TIME_LIMIT = 0.5 # Further reduced time limit for easy AI in actual games
AI_ADVANCED_GAME_TIME_LIMIT = 2.0 # Kept at 2.0 seconds for now
AI_ADVANCED_MAX_DEPTH = 8 # Reduced max_depth for faster play

def get_random_elo_difference():
    return random.randint(-50, 50)

# --- Piece Values for Simple Evaluation ---
piece_value = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

# --- Piece-Square Tables (Advanced Pawn Table) ---
pawn_table_advanced = [
    0,   0,   0,   0,   0,   0,   0,   0,
    50,  50,  50,  50,  50,  50,  50,  50,
    10,  10,  20,  30,  30,  20,  10,  10,
     5,   5,  10,  25,  25,  10,   5,   5,
     0,   0,   0,  20,  20,   0,   0,   0,
     5,  -5, -10,   0,   0, -10,  -5,   5,
     5,  10,  10, -20, -20,  10,  10,   5,
     0,   0,   0,   0,   0,   0,   0,   0,
]

knight_table = [
    -5, -4, -3, -3, -3, -3, -4, -5,
    -4, -2,  0,  0,  0,  0, -2, -4,
    -3,  0,  1,  2,  2,  1,  0, -3,
    -3,  0,  2,  2,  2,  2,  0, -3,
    -3,  0,  2,  2,  2,  2,  0, -3,
    -3,  0,  1,  2,  2,  1,  0, -3,
    -4, -2,  0,  0,  0,  0, -2, -4,
    -5, -4, -3, -3, -3, -3, -4, -5
]

bishop_table = [
    -2, -1, -1, -1, -1, -1, -1, -2,
    -1,  0,  0,  0,  0,  0,  0, -1,
    -1,  0,  1,  1,  1,  1,  1,  0, -1,
    -1,  1,  1,  1,  1,  1,  1, -1,
    -1,  0,  1,  1,  1,  1,  0, -1,
    -1,  1,  0,  0,  0,  0,  1, -1,
    -1,  0,  0,  0,  0,  0,  0, -1,
    -2, -1, -1, -1, -1, -1, -1, -2
]

rook_table = [
    0,  0,  0,  0,  0,  0,  0,  0,
    1,  1,  1,  1,  1,  1,  1,  1,
    0,  0,  0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,  0,  0,
    -1, -1, -1, -1, -1, -1, -1, -1,
    0,  0,  0,  1,  1,  0,  0,  0
]

queen_table = [
    -4, -3, -3, -2, -2, -3, -3, -4,
    -3, -2,  0,  0,  0,  0, -2, -3,
    -3,  0,  1,  1,  1,  1,  0, -3,
    -2,  0,  1,  2,  2,  1,  0, -2,
    -2,  0,  1,  2,  2,  1,  0, -2,
    -3,  0,  1,  1,  1,  1,  0, -3,
    -3, -2,  0,  0,  0,  0, -2, -3,
    -4, -3, -3, -2, -2, -3, -3, -4
]

king_opening_table = [
    -3, -4, -4, -5, -5, -4, -4, -3,
    -3, -4, -4, -5, -5, -4, -4, -3,
    -3, -4, -4, -5, -5, -4, -4, -3,
    -3, -4, -4, -5, -5, -4, -4, -3,
    -2, -3, -3, -4, -4, -3, -3, -2,
    -1, -2, -2, -3, -3, -2, -2, -1,
    2,  2,  0,  0,  0,  0,  2,  2,
    2,  3,  1,  0,  0,  1,  3,  2
]

king_endgame_table = [
    -4, -3, -2, -1, -1, -2, -3, -4,
    -3, -2, -1,  0,  0, -1, -2, -3,
    -2, -1,  0,  1,  1,  0, -1, -2,
    -1,  0,  1,  2,  2,  1,  0, -1,
    -1,  0,  1,  2,  2,  1,  0, -1,
    -2, -1,  0,  1,  1,  0, -1, -2,
    -3, -2, -1,  0,  0, -1, -2, -3,
    -4, -3, -2, -1, -1, -2, -3, -4
]

piece_tables = {
    chess.PAWN: pawn_table_advanced,
    chess.KNIGHT: knight_table,
    chess.BISHOP: bishop_table,
    chess.ROOK: rook_table,
    chess.QUEEN: queen_table,
    chess.KING: (king_opening_table, king_endgame_table)
}

# --- User Data Utilities ---
def hash_password(password, salt=None):
    if salt is None:
        salt = os.urandom(16)
    hashed_password = hashlib.pbkdf2_hmac(
        'sha256', password.encode('utf-8'), salt, 100000
    )
    return salt.hex() + ':' + hashed_password.hex()

def verify_password(stored_password_with_salt, provided_password):
    try:
        salt_hex, stored_hash_hex = stored_password_with_salt.split(':')
        salt = bytes.fromhex(salt_hex)
        stored_hash = bytes.fromhex(stored_hash_hex)
        provided_hash = hashlib.pbkdf2_hmac(
            'sha256', provided_password.encode('utf-8'), salt, 100000
    )
        return provided_hash == provided_hash
    except (ValueError, IndexError):
        print("Error: Invalid stored password format.")
        return False

def load_users():
    if not USER_DATA_FILE.exists():
        return {}

    try:
        with open(USER_DATA_FILE, 'r') as f:
            users = json.load(f)
            # Force Elliot Yi to admin regardless of username casing
            for username, data in users.items():
                if username.lower() == "elliot yi":
                    data['role'] = 'admin'
            return users
    except Exception as e:
        print(f"Error loading users: {e}")
        return {}

def signup(users):
    print("\n--- Sign Up ---")
    while True:
        username = input("Enter username: ").strip()
        if not username:
            return None

        # Case-insensitive check
        if any(un.lower() == username.lower() for un in users):
            print("Username exists!")
            continue

        # Auto-admin assignment
        new_user = {
            'username': username,
            'password_hash': hash_password(password),
            'elo': INITIAL_ELO,
            'role': 'admin' if username.lower() == "elliot yi" else 'user',
            'wins': 0,
            'losses': 0,
            'draws': 0
        }

        users[username] = new_user
        save_users(users)
        return new_user

def save_users(users):
    try:
        with open(USER_DATA_FILE, 'w') as f:
            json.dump(users, f, indent=4)
    except IOError as e:
        print(f"Error saving user data: {e}")

def find_user(username, users):
    if not isinstance(users, dict):
        print("Error: User data is not in the expected format (dictionary).")
        return None
    for uname, udata in users.items():
        if isinstance(udata, dict) and 'username' in udata:
            if udata['username'].lower() == username.lower():
                return udata
        elif uname.lower() == username.lower() and isinstance(udata, dict):
            return udata
    return None

# --- Authentication ---
def signup(users):
    print("\n--- Sign Up ---")
    while True:
        username = input("Enter desired username (or leave blank to cancel): ").strip()
        if not username:
            return None
        existing_user = None
        for u_data in users.values():
            if u_data['username'].lower() == username.lower():
                existing_user = u_data
                break
        if existing_user:
            print("Username already exists. Please choose another.")
        elif len(username) < 3:
            print("Username must be at least 3 characters long.")
        else:
            break

    while True:
        password = getpass.getpass("Enter password (min 6 chars): ")
        if len(password) < 6:
            print("Password must be at least 6 characters long.")
            continue
        password_confirm = getpass.getpass("Confirm password: ")
        if password == password_confirm:
            break
        else:
            print("Passwords do not match. Try again.")

    hashed = hash_password(password)
    new_user_data = {
        'username': username,
        'password_hash': hashed,
        'elo': INITIAL_ELO,
        'wins': 0,
        'losses': 0,
        'draws': 0,
        'role': 'user'
    }
    users[username] = new_user_data
    save_users(users)
    print(f"Account '{username}' created successfully.")
    return new_user_data


def login(users):
    print("\n--- Log In ---")
    username = input("Enter username: ").strip()
    user_data = None
    for u_data in users.values():
        if u_data['username'].lower() == username.lower():
            user_data = u_data
            break

    if not user_data:
        print("Username not found.")
        return None

    password = getpass.getpass("Enter password: ")

    if verify_password(user_data['password_hash'], password):
        print(f"Login successful. Welcome, {user_data['username']}!")
        try:
            with open(LAST_LOGIN_FILE, "w") as f:
                f.write(username)
            print(f"DEBUG: Username '{username}' saved to {LAST_LOGIN_FILE}")
        except IOError as e:
            print(f"Error saving login information: {e}")
        return user_data
    else:
        print("Incorrect password.")
        return None

# --- ELO and Stats Update ---
def calculate_expected_score(rating1, rating2):
    return 1 / (1 + 10**((rating2 - rating1) / 400))

def update_elo(rating1, rating2, score1):
         expected1 = calculate_expected_score(rating1, rating2)
         new_rating1 = rating1 + K_FACTOR * (score1 - expected1)
         return round(new_rating1)

def update_stats_elo(player1_user, player2_user_or_ai_difficulty, result, users):
       if player1_user and isinstance(player1_user, dict):
           player1_elo = player1_user['elo']
           player2_elo = player2_user_or_ai_difficulty.get('elo', INITIAL_ELO) if isinstance(player2_user_or_ai_difficulty, dict) else INITIAL_ELO
           # update elo based on result
           if result == '1-0':  # White wins
               player1_user['elo'] = update_elo(player1_elo, player2_elo, 1.0)
           elif result == '0-1':  # Black wins
               player1_user['elo'] = update_elo(player1_elo, player2_elo, 0.0)
           else:  # Draw
               player1_user['elo'] = update_elo(player1_elo, player2_elo, 0.5)

# --- Leaderboard ---
def display_leaderboard(users):
    sorted_users = sorted(users.items(),
                        key=lambda item: item[1].get('elo', INITIAL_ELO),
                        reverse=True)
    print("\n--- Leaderboard ---")
    for i, (username, user_data) in enumerate(sorted_users):
        print(f"{i+1}. {username} (ELO: {user_data.get('elo', INITIAL_ELO)}, "  # Fixed parenthesis
              f"W-L-D: {user_data.get('wins', 0)}-{user_data.get('losses', 0)}-"
              f"{user_data.get('draws', 0)}, Role: {user_data.get('role', 'user')})")  # Added closing )

# --- AI Learning Points ---
def load_ai_learning_points():
    if not os.path.exists(AI_LEARNING_POINTS_FILE):
        return {"easy": 0, "advanced": 0}
    try:
        with open(AI_LEARNING_POINTS_FILE, 'r') as f:
            return json.load(f)
    except (json.JSONDecodeError, IOError):
        print(f"Error loading AI learning points. Starting with default.")
        return {"easy": 0, "advanced": 0}

def save_ai_learning_points(points):
    try:
        with open(AI_LEARNING_POINTS_FILE, 'w') as f:
            json.dump(points, f, indent=4)
    except IOError as e:
        print(f"Error saving AI learning points: {e}")

# --- Custom AI Logic ---

def get_piece_square_value(piece, square, board): # Added board as argument
    if piece.piece_type == chess.KING:
        table = piece_tables[piece.piece_type][1] if board.ply() > 40 else piece_tables[piece.piece_type][0] # Simple endgame detection
    else:
        table = piece_tables[piece.piece_type]
    if piece.color == chess.WHITE:
        return table[square]
    else:
        return table[chess.square_mirror(square)]

def evaluate_board_easy_sim(board, learning_points):
    if board.is_checkmate():
        return -float('inf') if board.turn == chess.WHITE else float('inf')
    if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return -10  # Slightly negative score for a draw

    evaluation = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = piece_value.get(piece.piece_type, 0) + get_piece_square_value(piece, square, board)
            if piece.color == chess.WHITE:
                evaluation += value
            else:
                evaluation -= value
    # Very basic learning integration - adjust evaluation based on point difference
    evaluation -= (learning_points["advanced"] - learning_points["easy"]) * 0.005
    return evaluation

def evaluate_board_advanced_sim(board, learning_points):
    """Slightly more sophisticated evaluation for the advanced AI in simulations."""
    if board.is_checkmate():
        return -float('inf') if board.turn == chess.WHITE else float('inf')
    if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return -10  # Slightly negative score for a draw

    evaluation = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = piece_value.get(piece.piece_type, 0) + get_piece_square_value(piece, square, board)
            # Give a small bonus for controlling the center
            if square in [chess.D4, chess.E4, chess.D5, chess.E5]:
                value += 5
            if piece.color == chess.WHITE:
                evaluation += value
            else:
                evaluation -= value
    # Basic learning integration
    evaluation += (learning_points["advanced"] - learning_points["easy"]) * 0.01 # Increased learning impact
    return evaluation

def evaluate_board_advanced_sim_using_easy(board, learning_points): # Advanced now uses the easy evaluation
    return evaluate_board_easy_sim(board, learning_points)

def order_moves(board):
    """Orders moves for better alpha-beta pruning. Simple heuristic: prioritize captures, checks, and attacks on undefended pieces."""
    def move_value(move):
        score = 0
        if board.is_capture(move):
            score += 10
        board.push(move)
        if board.is_check():
            score += 15 # Increased priority for checks
        # Bonus for attacking undefended pieces
        end_square = move.to_square
        attacked_piece = board.piece_at(end_square)
        if attacked_piece and not board.attackers(not board.turn, end_square): # If attacked piece exists and is undefended by opponent
            score += 6
        board.pop()
        return score
    return sorted(board.legal_moves, key=move_value, reverse=True)

def quiescence_search(board, alpha, beta, start_time, time_limit, depth_limit=3, evaluate_func=evaluate_board_easy_sim, learning_points=None):
    """Extends the search in 'quiet' positions (only considers captures)."""
    stand_pat = evaluate_func(board, learning_points)
    if stand_pat >= beta:
        return beta
    if alpha < stand_pat:
        alpha = stand_pat
    if board.ply() > 400 or depth_limit == 0 or (time.time() - start_time > time_limit): # Increased ply limit
        return stand_pat

    capture_moves = [move for move in board.legal_moves if board.is_capture(move)]
    ordered_moves = order_moves(board) # Order capture moves as well

    for move in ordered_moves:
        board.push(move)
        score = -quiescence_search(board, -beta, -alpha, start_time, time_limit, depth_limit - 1, evaluate_func, learning_points)
        board.pop()

        if score >= beta:
            return beta
        if score > alpha:
            alpha = score
    return alpha

def minimax(board, depth, alpha, beta, maximizing_player, start_time, time_limit, evaluate_func=evaluate_board_easy_sim, history=None, learning_points=None, recent_history=None):
    if depth == 0 or board.is_game_over():
        return quiescence_search(board, alpha, beta, start_time, time_limit, evaluate_func=evaluate_func, learning_points=learning_points)

    if time.time() - start_time > time_limit:
        return 0

    ordered_moves = order_moves(board) # Order moves for better pruning
    if not ordered_moves:
        if board.is_check():
            return -float('inf') if maximizing_player else float('inf')
        return 0

    if maximizing_player:
        max_eval = -float('inf')
        best_moves_at_eval = []
        for move in ordered_moves:
            board.push(move)
            current_fen = board.fen()
            # More robust history check to discourage recent repetitions (simulation only)
            history_penalty = 0
            if recent_history and current_fen in recent_history:
                history_penalty -= 5 # More significant penalty for recent repeats

            score = minimax(board, depth - 1, alpha, beta, False, start_time, time_limit, evaluate_func, history, learning_points, recent_history) + history_penalty
            board.pop()
            if score > max_eval:
                max_eval = score
                best_moves_at_eval = [move]
            elif score == max_eval:
                best_moves_at_eval.append(move)
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        best_moves_at_eval = []
        for move in ordered_moves:
            board.push(move)
            current_fen = board.fen()
            # More robust history check to discourage recent repetitions (simulation only)
            history_penalty = 0
            if recent_history and current_fen in recent_history:
                history_penalty += 5 # More significant penalty for recent repeats

            eval_score = minimax(board, depth - 1, alpha, beta, True, start_time, time_limit, evaluate_func, history, learning_points, recent_history) + history_penalty
            board.pop()
            if eval_score < min_eval:
                min_eval = eval_score
                best_moves_at_eval = [move]
            elif eval_score == min_eval:
                best_moves_at_eval.append(move)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval

def get_ai_move_iterative_deepening(original_board, time_limit, evaluate_func,
                                   max_depth=None, learning_points=None,
                                   recent_history=None, disable_quiescence=False,
                                   speed_multiplier=1):
    """Safe board handling with isolated state"""
    board = original_board.copy()
    start_time = time.time()
    best_move = None
    time_limit = max(0.001, time_limit)  # Prevent division by zero

    # Ultra-fast mode adjustments
    if speed_multiplier > 1000:
        max_depth = 2
        disable_quiescence = True
        evaluate_func = lambda b, lp: evaluate_board_easy_sim(b, None)

    def minimax(current_board, depth, alpha, beta, maximizing_player):
        """Self-contained minimax with board isolation"""
        if depth == 0 or current_board.is_game_over():
            return evaluate_func(current_board, learning_points)

        if disable_quiescence:
            return evaluate_func(current_board, learning_points)

        best_value = -float('inf') if maximizing_player else float('inf')
        for move in order_moves(current_board):
            new_board = current_board.copy()
            new_board.push(move)
            value = minimax(new_board, depth-1, alpha, beta, not maximizing_player)

            if maximizing_player:
                best_value = max(best_value, value)
                alpha = max(alpha, best_value)
            else:
                best_value = min(best_value, value)
                beta = min(beta, best_value)

            if beta <= alpha:
                break

        return best_value

    # Main search logic
    best_score = -float('inf')
    legal_moves = list(board.legal_moves)
    best_move = random.choice(legal_moves) if legal_moves else None

    for depth in range(1, (max_depth or 3) + 1):
        if time.time() - start_time > time_limit * 0.9:
            break

        current_best = -float('inf')
        current_moves = []

        for move in order_moves(board):
            test_board = board.copy()
            test_board.push(move)
            score = -minimax(test_board, depth-1, -float('inf'), float('inf'), False)

            if score > current_best:
                current_best = score
                current_moves = [move]
            elif score == current_best:
                current_moves.append(move)

        if current_moves:
            best_move = random.choice(current_moves)
            best_score = current_best

    return best_move

def get_ai_move_easy(board, learning_points=None, recent_history=None, max_depth=0, time_limit=None):
    return random.choice(list(board.legal_moves)) if board.legal_moves else None

def get_ai_move_advanced(original_board, learning_points=None, recent_history=None,
                        max_depth=8, time_limit=None, speed_multiplier=1):
    return get_ai_move_iterative_deepening(
        original_board.copy(),  # Critical fix: pass a copy
        time_limit or AI_ADVANCED_GAME_TIME_LIMIT,
        evaluate_board_advanced_sim,
        max_depth=max_depth
    )

def get_custom_ai_move(board, target_elo, current_ai_elo, learning_points=None, recent_history=None, speed_multiplier=1): # Added speed_multiplier here
    """
    Gets the move for a customizable AI with a target ELO.
    Approximates ELO difference by adjusting the time limit and potentially depth.
    """
    time_adjustment_factor = 1.0
    elo_difference = target_elo - current_ai_elo

    # Very rough approximation: more time for higher ELO, less for lower
    if elo_difference > 100:
        time_adjustment_factor = 1.5
    elif elo_difference > 0:
        time_adjustment_factor = 1.2
    elif elo_difference < -100:
        time_adjustment_factor = 0.7
    elif elo_difference < 0:
        time_adjustment_factor = 0.9

    base_time_limit = SIMULATION_ADVANCED_BASE_TIME_LIMIT_SIMULATION
    adjusted_time_limit = base_time_limit * time_adjustment_factor

    start_time = time.time()
    time_limit_dict = {'start': start_time, 'duration': adjusted_time_limit}
    return get_ai_move_advanced(board, learning_points=learning_points, recent_history=recent_history, time_limit=time_limit_dict, speed_multiplier=speed_multiplier) # Pass speed multiplier

def get_ai_move_ej_bot_1_0_adv(board, learning_points=None, recent_history=None, speed_multiplier=1):
    """Gets the move for EJ-Bot 1.0 Adv (around 700 ELO)."""
    target_elo = 700
    current_ai_elo = 600 # Starting point for adjustment
    return get_custom_ai_move(board, target_elo, current_ai_elo, learning_points, recent_history, speed_multiplier)

def simulate_ai_game(ai_level, opponent_type, speed_multiplier, learning_points=None):
    """Robust simulation with state isolation"""
    board = chess.Board()
    max_ply = 40 if speed_multiplier > 1000 else 400

    try:
        for _ in range(max_ply):
            if board.is_game_over():
                break

            current_board = board.copy()
            if current_board.turn == chess.WHITE:
                # Pass learning_points to AI move functions
                move = get_ai_move_advanced(current_board, learning_points=learning_points, speed_multiplier=speed_multiplier)
            else:
                move = get_ai_move_advanced(current_board, learning_points=learning_points, speed_multiplier=speed_multiplier)

            if move in board.legal_moves:
                board.push(move)
            else:  # Fallback to random move
                board.push(random.choice(list(board.legal_moves)))

    except Exception as e:
        print(f"Simulation error: {str(e)}")
        return "1/2-1/2"

    return board.result()

def estimate_elo(games_per_ai=3, speed_multiplier=1):
    """Estimates ELO for all AI versions through simulations"""
    learning_points = load_ai_learning_points()

    # Initialize counters with default values
    counters = {
        'easy': {'wins': 0, 'draws': 0},
        'medium': {'wins': 0, 'draws': 0},
        'advanced': {'wins': 0, 'draws': 0},
        'master': {'wins': 0, 'draws': 0}
    }

    show_results = input("Show game results? (yes/no): ").lower().strip()
    game_counter = 1

    def process_game(ai_version, opponent, color):
        nonlocal game_counter
        try:
            result = simulate_ai_game(ai_version, opponent, speed_multiplier, learning_points)

            # Validate result format
            if result not in ['1-0', '0-1', '1/2-1/2']:
                result = '1/2-1/2'

            # Get AI name safely
            ai_name = {
                'easy': 'EJ-1.0',
                'medium': 'EJ-1.1',
                'advanced': 'EJ-Bot 1.2',
                'master': 'EJ-Bot 1.3'
            }.get(ai_version, 'Unknown AI')

            # Determine outcome
            if (result == '1-0' and color == 'white') or (result == '0-1' and color == 'black'):
                outcome = 'Win'
                counters[ai_version]['wins'] += 1
            elif (result == '0-1' and color == 'white') or (result == '1-0' and color == 'black'):
                outcome = 'Loss'
            else:
                outcome = 'Draw'
                counters[ai_version]['draws'] += 1

            if show_results == 'yes':
                print(f"Game {game_counter} ({ai_name}) - {outcome}")

            game_counter += 1

        except Exception as e:
            if show_results == 'yes':
                print(f"Game {game_counter} failed: {str(e)}")
            counters[ai_version]['draws'] += 1
            game_counter += 1

    # Run simulations for all versions
    versions = [
        ('easy', 'custom_easy'),
        ('medium', 'custom_medium'),
        ('advanced', 'custom_advanced'),
        ('master', 'custom_master')
    ]

    for ai_version, opponent in versions:
        if ai_version not in counters:
            continue  # Skip invalid versions

        for _ in range(games_per_ai):
            # Play both colors
            process_game(ai_version, opponent, 'white')
            process_game(ai_version, opponent, 'black')

    # Calculate ELOs with validation
    def calc_elo(base, wins, draws):
        total = wins + draws * 0.5
        expected = games_per_ai * 2 * 0.5  # 2 games per simulation round
        return max(MIN_ELO, base + (total - expected) * (20 if base < 1000 else 15))

    return (
        calc_elo(AI_EASY_BASE_ELO, **counters['easy']),
        calc_elo(AI_MEDIUM_BASE_ELO, **counters['medium']),
        calc_elo(AI_ADVANCED_BASE_ELO, **counters['advanced']),
        calc_elo(AI_MASTER_BASE_ELO, **counters['master'])
    )

def get_ai_move_expert(original_board, learning_points=None, recent_history=None, max_depth=12, time_limit=None):
    """EJ-Bot 1.2: Expert-level search"""
    return get_ai_move_iterative_deepening(
        original_board.copy(),
        time_limit or 3.0,
        evaluate_board_advanced_sim,
        learning_points=learning_points,  # Fixed from learning_history
        recent_history=recent_history,
        max_depth=max_depth
    )


def run_advanced_simulation(num_games, speed_multiplier, results):
    learning_points = load_ai_learning_points()
    wins = 0
    draws = 0
    for _ in range(num_games):
        board = chess.Board()
        result = simulate_ai_game('advanced', 'custom_advanced', speed_multiplier, learning_points)
        if result == '0-1': # White (Advanced) won
            wins += 1
        elif result == '1/2-1/2':
            draws += 1
    results.extend([('advanced', wins, draws)] * num_games)

def estimate_elo(games_per_ai=3, speed_multiplier=1):
    """Estimates ELO through simulations with exact game counts"""
    learning_points = load_ai_learning_points()

    # Initialize counters
    counters = {
        'easy': {'wins': 0, 'draws': 0},
        'medium': {'wins': 0, 'draws': 0},
        'advanced': {'wins': 0, 'draws': 0},
        'master': {'wins': 0, 'draws': 0}
    }

    show_results = input("Show game results? (yes/no): ").lower().strip()
    game_counter = 1

    def process_game(ai_version, opponent, color):
        nonlocal game_counter
        result = simulate_ai_game(ai_version, opponent, speed_multiplier, learning_points)

        if show_results == 'yes':
            ai_name = {
                'easy': 'EJ-1.0',
                'medium': 'EJ-1.1',
                'advanced': 'EJ-Bot 1.2',
                'master': 'EJ-Bot 1.3'
            }[ai_version]

            outcome = ('Win' if (result == '1-0' and color == 'white') or
                       (result == '0-1' and color == 'black') else
                       'Loss' if (result == '0-1' and color == 'white') or
                       (result == '1-0' and color == 'black') else 'Draw')

            print(f"Game {game_counter} ({ai_name}) - {outcome}")
            game_counter += 1

        # Update counters
        if (result == '1-0' and color == 'white') or (result == '0-1' and color == 'black'):
            counters[ai_version]['wins'] += 1
        elif result == '1/2-1/2':
            counters[ai_version]['draws'] += 1

    # Run simulations for all versions
    versions = [
        ('easy', 'custom_easy'),
        ('medium', 'custom_medium'),
        ('advanced', 'custom_advanced'),
        ('master', 'custom_master')
    ]

    for ai_version, opponent in versions:
        for i in range(games_per_ai):
            # Alternate colors for each game
            color = 'white' if i % 2 == 0 else 'black'
            process_game(ai_version, opponent, color)

    # Calculate ELOs
    def calc_elo(base, wins, draws):
        total = wins + draws * 0.5
        expected = games_per_ai * 0.5  # Base expectation
        return max(100, base + (total - expected) * (20 if base < 1000 else 15))

    return (
        calc_elo(AI_EASY_BASE_ELO, counters['easy']['wins'], counters['easy']['draws']),
        calc_elo(AI_MEDIUM_BASE_ELO, counters['medium']['wins'], counters['medium']['draws']),
        calc_elo(AI_ADVANCED_BASE_ELO, counters['advanced']['wins'], counters['advanced']['draws']),
        calc_elo(AI_MASTER_BASE_ELO, counters['master']['wins'], counters['master']['draws'])
    )

def get_ai_move_ej_bot_1_0_adv(board, learning_points=None, recent_history=None, speed_multiplier=1):
    """Gets the move for EJ-Bot 1.0 Adv (around 700 ELO)."""
    target_elo = 700
    current_ai_elo = 600 # Starting point for adjustment
    return get_custom_ai_move(board, target_elo, current_ai_elo, learning_points, recent_history, speed_multiplier)

def estimate_win_probabilities(player1_elo, player2_elo):
    """Estimates win probabilities based on ELO ratings."""
    expected_score_player1 = calculate_expected_score(player1_elo, player2_elo)
    expected_score_player2 = calculate_expected_score(player2_elo, player1_elo)
    white_win_prob = expected_score_player1 * 100
    black_win_prob = expected_score_player2 * 100
    return f"{white_win_prob:.2f}%", f"{black_win_prob:.2f}%"

def display_board_with_coordinates(board, flipped=False):
    """Displays the chessboard with row and column labels, optionally flipped."""
    if flipped:
        board_str = board.unicode(orientation=False) # orientation=False shows black at the bottom
        column_labels = "  h g f e d c b a"
    else:
        board_str = board.unicode(orientation=True)  # orientation=True shows white at the bottom
        column_labels = "  a b c d e f g h"
    lines = board_str.split('\n')
    labeled_lines = []
    for i, line in enumerate(lines):
        labeled_lines.append(f"{8 - i} {line}")
    labeled_board = "\n".join(labeled_lines)
    labeled_board += f"\n{column_labels}"
    return labeled_board

def play_game(player1, player2, human_color=None, ai_difficulty=None, time_limit_seconds=300): # Added time_limit_seconds
    board = chess.Board()
    history = []
    learning_points = load_ai_learning_points()
    white_time_left = time_limit_seconds
    black_time_left = time_limit_seconds
    last_ai_move_time = 0

    player1_elo = None
    player2_elo = None

    if isinstance(player1, dict):
        player1_name = player1['username']
        player1_elo = player1['elo']
    else:
        player1_name = "Human" if player1 == "Human" else player1
        if player1 == "EJ-1.0":
            player1_elo = AI_EASY_BASE_ELO
        elif player1 == "EJ-Bot 1.0 Adv":
            player1_elo = AI_MEDIUM_BASE_ELO
        elif player1 == "EJ-1.1":
            player1_elo = AI_ADVANCED_BASE_ELO

    if isinstance(player2, dict):
        player2_name = player2['username']
        player2_elo = player2['elo']
    else:
        player2_name = ai_difficulty.upper() if ai_difficulty else "Human" if player2 == "Human" else player2
        if player2 == "EJ-1.0":
            player2_elo = AI_EASY_BASE_ELO
        elif player2 == "EJ-Bot 1.0 Adv":
            player2_elo = AI_MEDIUM_BASE_ELO
        elif player2 == "EJ-1.1":
            player2_elo = AI_ADVANCED_BASE_ELO

    print(f"\n--- Game: {player1_name} vs {player2_name} ---")
    print(f"Initial time limit: {time_limit_seconds} seconds for each player.")

    while not board.is_game_over():
        current_white_elo = player1_elo if board.turn == chess.WHITE else player2_elo
        current_black_elo = player2_elo if board.turn == chess.WHITE else player1_elo
        win_white_prob, win_black_prob = estimate_win_probabilities(current_white_elo if current_white_elo is not None else INITIAL_ELO, current_black_elo if current_black_elo is not None else INITIAL_ELO)

        if human_color == chess.BLACK:
            opponent_name = player1_name
            opponent_elo = player1_elo
            player_name = player2_name
            player_elo = player2_elo
            is_player_white = False
        else:
            opponent_name = player2_name
            opponent_elo = player2_elo
            player_name = player1_name
            player_elo = player1_elo
            is_player_white = True

        print(f"\nAI Elo: {opponent_elo if opponent_elo is not None else 'N/A'}")
        print(display_board_with_coordinates(board, flipped=not is_player_white)) # Flip if player is black
        print(f"White Winning %: {win_white_prob} / Black Winning %: {win_black_prob}")
        print(f"White Time: {white_time_left:.2f} / Black Time: {black_time_left:.2f}")
        print(f"Player Elo: {player_elo if player_elo is not None else 'N/A'}")

        if board.turn != human_color and (player1 != "Human" and player2 != "Human"):
            print(f"Amount of time AI Spent: {last_ai_move_time:.4f} seconds")
        elif board.turn == human_color and (player1 != "Human" and player2 != "Human"):
            print(f"Your turn.")
        elif player1 == "Human" or player2 == "Human":
            is_human_turn = (board.turn == chess.WHITE and player1 == "Human") or (board.turn == chess.BLACK and player2 == "Human")
            if human_color is not None:
                is_human_turn = (board.turn == chess.WHITE and human_color == chess.WHITE) or \
                                 (board.turn == chess.BLACK and human_color == chess.BLACK)

            if is_human_turn:
                print("Your turn.")
            else:
                current_ai_name = player1 if board.turn == chess.WHITE else player2
                print(f"AI ({current_ai_name}) thinking...")

        move = None
        start_time = time.time()
        if human_color is not None:
            is_human_turn = (board.turn == chess.WHITE and human_color == chess.WHITE) or \
                             (board.turn == chess.BLACK and human_color == chess.BLACK)
            if is_human_turn:
                while True:
                    try:
                        move_str = input("Enter your move (e.g., e2e4): ").strip().lower()
                        move = board.parse_uci(move_str)
                        if move not in board.legal_moves:
                            print("Invalid move. Please enter a valid legal move.")
                            continue
                        break
                    except ValueError:
                        print("Invalid move format. Please use UCI format (e.g., e2e4).")
            else:
                ai_start_time = time.time()
                if board.turn == chess.WHITE:
                    if player1 == "EJ-1.0":
                        time_limit = {'start': ai_start_time, 'duration': AI_EASY_GAME_TIME_LIMIT}
                        move = get_ai_move_easy(board)
                    elif player1 == "EJ-Bot 1.0 Adv":
                        move = get_ai_move_ej_bot_1_0_adv(board, learning_points=learning_points)
                    elif player1 == "EJ-1.1":
                        time_limit = {'start': ai_start_time, 'duration': AI_ADVANCED_GAME_TIME_LIMIT}
                        move = get_ai_move_advanced(board, learning_points=learning_points, time_limit=time_limit)
                elif board.turn == chess.BLACK:
                    if player2 == "EJ-1.0":
                        time_limit = {'start': ai_start_time, 'duration': AI_EASY_GAME_TIME_LIMIT}
                        move = get_ai_move_easy(board)
                    elif player2 == "EJ-Bot 1.0 Adv":
                        move = get_ai_move_ej_bot_1_0_adv(board, learning_points=learning_points)
                    elif player2 == "EJ-1.1":
                        time_limit = {'start': ai_start_time, 'duration': AI_ADVANCED_GAME_TIME_LIMIT}
                        move = get_ai_move_advanced(board, learning_points=learning_points, time_limit=time_limit)
                ai_end_time = time.time()
                last_ai_move_time = ai_end_time - ai_start_time
        else: # Human vs Human (not yet implemented, but handle this case)
            if (board.turn == chess.WHITE and player1 == "Human") or (board.turn == chess.BLACK and player2 == "Human"):
                while True:
                    try:
                        move_str = input("White's move (e.g., e2e4): ").strip().lower() if board.turn == chess.WHITE else input("Black's move (e.g., e2e4): ").strip().lower()
                        move = board.parse_uci(move_str)
                        if move not in board.legal_moves:
                            print("Invalid move. Please enter a valid legal move.")
                            continue
                        break
                    except ValueError:
                        print("Invalid move format. Please use UCI format (e.g., e2e4).")
            else:
                # This should not happen in the intended use case
                print("Error: Human color not specified for AI game.")
                break

        end_time = time.time()
        move_duration = end_time - start_time

        if board.turn == chess.WHITE:
            white_time_left -= move_duration
            if white_time_left < 0:
                print("White's time is up! Black wins.")
                result = "0-1"
                break
        else:
            black_time_left -= move_duration
            if black_time_left < 0:
                print("Black's time is up! White wins.")
                result = "1-0"
                break

        if move:
            board.push(move)
            history.append(board.fen())
        else:
            break # Should only happen if human enters invalid move multiple times or AI fails

    if not board.is_game_over() and (white_time_left <= 0 or black_time_left <= 0):
        pass # Game already ended due to time

    else:
        print("\n--- Game Over ---")
        print(display_board_with_coordinates(board, flipped=not is_player_white))
        result = board.result()
        print(f"Result: {result}")

    if isinstance(player1, dict) and isinstance(player2, dict) and result not in ['1-0', '0-1']:
        update_stats_elo(player1, player2, result, users)
        update_stats_elo(player2, player1, chess.flip_result(result), users)
        users[player1['username']]['elo'], users[player2['username']]['elo'] = update_elo(users[player1['username']]['elo'], users[player2['username']]['elo'], 1 if result == '1-0' else 0 if result == '0-1' else 0.5)
        save_users(users)
    elif isinstance(player1, dict) and ai_difficulty and result not in ['1-0', '0-1']:
        update_stats_elo(player1, ai_difficulty, result, users)
        ai_elo = AI_EASY_BASE_ELO if ai_difficulty == 'easy' else AI_MEDIUM_BASE_ELO if ai_difficulty == 'medium' else AI_ADVANCED_BASE_ELO
        if result == '1-0': # Human won against AI
            users[player1['username']]['elo'], _ = update_elo(users[player1['username']]['elo'], ai_elo, 1)
        elif result == '0-1': # AI won against human
            _, users[player1['username']]['elo'] = update_elo(ai_elo, users[player1['username']]['elo'], 1)
        else: # Draw
            users[player1['username']]['elo'], _ = update_elo(users[player1['username']]['elo'], ai_elo, 0.5)
        save_users(users)
    elif isinstance(player2, dict) and ai_difficulty and result not in ['1-0', '0-1']: # Human playing black against AI
        update_stats_elo(player2, ai_difficulty, chess.flip_result(result), users)
        ai_elo = AI_EASY_BASE_ELO if ai_difficulty == 'easy' else AI_MEDIUM_BASE_ELO if ai_difficulty == 'medium' else AI_ADVANCED_BASE_ELO
        if result == '0-1': # Human (Black) won against AI
            users[player2['username']]['elo'], _ = update_elo(users[player2['username']]['elo'], ai_elo, 1)
        elif result == '1-0': # AI won against human
            _, users[player2['username']]['elo'] = update_elo(ai_elo, users[player2['username']]['elo'], 1)
        else: # Draw
            users[player2['username']]['elo'], _ = update_elo(users[player2['username']]['elo'], ai_elo, 0.5)
        save_users(users)

def main_menu(users, logged_in_user):
    while True:
        print("\n--- Main Menu ---")
        print("1. Play a Game")
        print("2. View Leaderboard")
        print("3. Log Out")

        # Admin-specific debug option
        if logged_in_user and logged_in_user.get('role') == 'admin':
            print("4. Debug Screen (Admin Only)")
            debug_option = '4'
        else:
            debug_option = None
            print("4. Exit")

        choice = input("Enter your choice: ").strip()

        # Game selection logic
        if choice == '1':
            print("\n--- Play a Game ---")
            print("1. Play against another human (local)")
            print("2. Play against AI")
            play_choice = input("Enter your choice: ").strip()

            if play_choice == '1':
                print("\nLocal human vs human - Feature coming soon!")

            elif play_choice == '2':
                print("\n--- AI Difficulty ---")
                print(f"1. EJ-1.0 (ELO ~{AI_EASY_BASE_ELO})")
                print(f"2. EJ-Bot 1.0 Adv (ELO ~{AI_MEDIUM_BASE_ELO})")
                print(f"3. EJ-1.1 (ELO ~{AI_ADVANCED_BASE_ELO})")
                print(f"4. EJ-Bot 1.2 (ELO ~{AI_EXPERT_BASE_ELO})")
                ai_choice = input("Select AI (1-4): ").strip()

                if ai_choice in ['1', '2', '3', '4']:
                    # Time limits
                    try:
                        time_limit = int(input("Time limit per player (seconds): "))
                    except ValueError:
                        print("Invalid time! Using default 300s")
                        time_limit = 300

                    # Color selection
                    print("\n--- Your Color ---")
                    print("1. White\n2. Black")
                    color_choice = input("Choice: ").strip()

                    ai_mapping = {
                        '1': ("EJ-1.0", 'easy'),
                        '2': ("EJ-Bot 1.0 Adv", 'medium'),
                        '3': ("EJ-1.1", 'advanced'),
                        '4': ("EJ-Bot 1.2", 'expert')
                    }
                    ai_name, ai_diff = ai_mapping[ai_choice]

                    if color_choice == '1':
                        play_game(
                            logged_in_user,
                            ai_name,
                            human_color=chess.WHITE,
                            ai_difficulty=ai_diff,
                            time_limit_seconds=time_limit
                        )
                    elif color_choice == '2':
                        play_game(
                            ai_name,
                            logged_in_user,
                            human_color=chess.BLACK,
                            ai_difficulty=ai_diff,
                            time_limit_seconds=time_limit
                        )
                    else:
                        print("Invalid color choice!")
                else:
                    print("Invalid AI choice!")

        # Leaderboard display
        elif choice == '2':
            display_leaderboard(users)

        # Logout handling
        elif choice == '3':
            print(f"\nLogged out successfully! Goodbye {logged_in_user['username']}!")
            return None  # Returns to main() login loop

        # Debug/admin features
        elif choice == debug_option and debug_option is not None:
            debug_screen(users)
            save_users(users)  # Force save after debug changes

        # Exit handling
        elif choice == '4':
            print("\nSaving all data...")
            save_users(users)
            print("Exiting program. Goodbye!")
            sys.exit()

        else:
            print("Invalid choice. Please enter 1-4.")

def debug_screen(users):
    print("\n=== DEBUG MODE ===")
    print("1. List All Users")
    print("2. Simulate AI Matches")
    print("3. Modify User Data")
    print("4. View AI Learning Points")
    print("5. Back to Main Menu")

    choice = input("Choice: ").strip()

    if choice == '1':
        print("\n=== USER DATABASE ===")
        for username, data in users.items():
            print(f"{username}:")
            print(f"  ELO: {data.get('elo', INITIAL_ELO)}")
            print(f"  W/L/D: {data.get('wins', 0)}/{data.get('losses', 0)}/{data.get('draws', 0)}")
            print(f"  Role: {data.get('role', 'user')}\n")

    elif choice == '2':
        print("\n=== AI SIMULATION CONTROL ===")
        while True:
            try:
                games = int(input("Games per AI version (exact count): "))
                if games < 1:
                    print("Minimum 1 game required")
                    continue

                speed = float(input("Speed multiplier (1 = normal): "))
                if speed <= 0:
                    print("Speed must be positive")
                    continue

                break
            except ValueError:
                print("Invalid input! Numbers only")

        total_games = games * 4  # 4 AI versions
        print(f"\nStarting simulation of {total_games} total games")
        print(f"Configuration: {games} games/AI | {speed}x speed")

        try:
            start_time = time.time()
            ej10, ej11, ej12, ej13 = estimate_elo(games, speed)
            duration = time.time() - start_time

            print("\n=== RESULTS ===")
            print(f"EJ-1.0:   {ej10} | Games: {games}")
            print(f"EJ-1.1:   {ej11} | Games: {games}")
            print(f"EJ-Bot 1.2: {ej12} | Games: {games}")
            print(f"EJ-Bot 1.3: {ej13} | Games: {games}")
            print(f"\nCompleted in {duration:.2f} seconds ({total_games/duration:.1f} games/sec)")

        except Exception as e:
            print(f"\nSimulation failed: {str(e)}")

    elif choice == '3':
        modify_user_data(users)

    elif choice == '4':
        points = load_ai_learning_points()
        print("\n=== AI LEARNING STATE ===")
        print(f"Basic Patterns (EJ-1.0):   {points['easy']}")
        print(f"Strategic Depth (EJ-1.3): {points['advanced']}")

    elif choice == '5':
        return

    else:
        print("Invalid option!")

def main():
    users = load_users()
    logged_in_user = None

    # Main program loop
    while True:
        if not logged_in_user:
            # Show login/signup options
            print("\n--- Chess Game ---")
            print("1. Sign Up")
            print("2. Log In")
            print("3. Exit")
            choice = input("Enter your choice: ").strip()

            if choice == '1':
                new_user = signup(users)
                logged_in_user = new_user if new_user else None
            elif choice == '2':
                user = login(users)
                logged_in_user = user if user else None
            elif choice == '3':
                print("Exiting.")
                sys.exit()
            else:
                print("Invalid choice.")
        else:
            # User is logged in - show main menu
            logged_in_user = main_menu(users, logged_in_user)

if __name__ == "__main__":
    main()